# First Objective: "Grotrian diagram visualisation"

This is my solution to the first objective of the "Grotrian diagram visualisation".
[GSoC'23 ideas page](https://tardis-sn.github.io/gsoc_2023/ideas/)

In [1]:
# Download the atomic data
from tardis.io.atom_data.util import download_atom_data
download_atom_data('kurucz_cd23_chianti_H_He')


Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [ ]:
# Run simulation
from tardis import run_tardis
sim = run_tardis('tardis_example.yml', virtual_packet_logging=True, log_level='CRITICAL')

# Si ion density vs velocity

In [3]:
# Define Si constants
SI_ATOMIC_NUMBER = 14
SI_SYMBOL = 'Si'

In [4]:
# Extract ion densities
si_ion_densities = {}
si_ion_densities[0] = sim.plasma.ion_number_density.loc[SI_ATOMIC_NUMBER, 0]
si_ion_densities[1] = sim.plasma.ion_number_density.loc[SI_ATOMIC_NUMBER, 1]

In [5]:
# Combined densities of all silicon ions (Used to normalize individual ion densities)
total_si_densities = sim.plasma.ion_number_density.loc[SI_ATOMIC_NUMBER].sum(axis=0)

In [7]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

# Get middle velocities
middle_velocities = sim.model.v_middle.si / 1000 # scale down by 1000 for km/s

# Set custom text for hover tooltip
text = [f'Shell no.: {i}' for i in range(len(middle_velocities))]
hovertemplate = 'Ion charge: 0<br>Velocity: %{x:.0f}<br>Density: %{y}<br>%{text}'

# Add plots for each ion
fig.add_trace(go.Scatter(x=middle_velocities, y=si_ion_densities[0] / total_si_densities, name="Neutral Si",
                    line_shape='hvh', hovertemplate=hovertemplate, text=text))
fig.add_trace(go.Scatter(x=middle_velocities, y=si_ion_densities[1] / total_si_densities, name="Singly Ionized Si",
                    line_shape='hvh', hovertemplate=hovertemplate, text=text))

# Update fig layout and metadata
fig.update_yaxes(type="log", title="Ion Density / Element Density")
fig.update_xaxes(title="Velocity (km/s)")
fig.update_traces(mode="lines+markers")
fig.update_layout(title_text="Si Fractional Ion Density vs Velocity", title_x=0.5,
                  legend=dict(y=0.5, traceorder="reversed", font_size=16))

fig.show()